In [12]:
import random
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
weight = [random.randint(30, 80) for i in range(0, 1000) ]
height = [random.randint(140, 200) for i in range(0, 1000) ]


In [5]:
pd.DataFrame({'weight': weight, 'height': height}).to_csv('data.csv', index=False)

In [6]:
df = pd.read_csv('data.csv')

In [7]:
df.head()

,weight,height
0,79,197
1,69,200
2,51,154
3,48,184
4,59,189
